In [1]:
import os
import pandas as pd
from pydub import AudioSegment
from tqdm import tqdm
from glob import glob

In [61]:
tsv_path = 'transctipt1.tsv'
df = pd.read_csv(tsv_path, sep='\t', header = None)
print(df)
df.columns = ['0', '1', '2']    

                 0      1                     2
0       4aa1fdca33  6a6d1      ००७ मिलको दूरीमा
1       86e521554a  1f680      ००७ मिलको दूरीमा
2       c96b2b1afb  e349f      ००७ मिलको दूरीमा
3       2f7ab41953  294d8  ०११ देखि काङ्ग्रेसको
4       c290e5124b  88901      ०१ सौर द्रव्यमान
...            ...    ...                   ...
157900  17c05d0bdb  06372      ह्रास हुन लाग्दछ
157901  2aa4a03723  79c24      ह्रास हुन लाग्दछ
157902  4cb59eba81  99866      ह्रास हुन लाग्दछ
157903  223df784fc  e7c22           ह्रास हुँदै
157904  fc3b1054b0  95358       ह्वाइटआई भनिन्छ

[157905 rows x 3 columns]


In [62]:
audio_root_dir = r'd:\nepali_notetaker_project\nepali_notetaker_package\DataSet\asr_nepali_1\asr_nepali\data\16'

flac_path = glob(os.path.join(audio_root_dir, '**', '*.flac'), recursive=True)

# file_path = (os.path.splitext(os.path.basename(f))[0] for f in flac_path)

print(flac_path)
file_map = {os.path.splitext(os.path.basename(f))[0]: f for f in flac_path}
print(file_map)
# Add full path to dataframe
df['flac_path'] = df['0'].map(file_map)

df['file_id'] = df['flac_path'].apply(lambda x: os.path.splitext(os.path.basename(x))[0] if pd.notnull(x) else None)
# Drop missing entries
df = df[df['flac_path'].notnull()]


['d:\\nepali_notetaker_project\\nepali_notetaker_package\\DataSet\\asr_nepali_1\\asr_nepali\\data\\16\\16002139b6.flac', 'd:\\nepali_notetaker_project\\nepali_notetaker_package\\DataSet\\asr_nepali_1\\asr_nepali\\data\\16\\1600e07981.flac', 'd:\\nepali_notetaker_project\\nepali_notetaker_package\\DataSet\\asr_nepali_1\\asr_nepali\\data\\16\\160172b0b8.flac', 'd:\\nepali_notetaker_project\\nepali_notetaker_package\\DataSet\\asr_nepali_1\\asr_nepali\\data\\16\\1601811082.flac', 'd:\\nepali_notetaker_project\\nepali_notetaker_package\\DataSet\\asr_nepali_1\\asr_nepali\\data\\16\\16018c784f.flac', 'd:\\nepali_notetaker_project\\nepali_notetaker_package\\DataSet\\asr_nepali_1\\asr_nepali\\data\\16\\1601a3c1e5.flac', 'd:\\nepali_notetaker_project\\nepali_notetaker_package\\DataSet\\asr_nepali_1\\asr_nepali\\data\\16\\1601af3da0.flac', 'd:\\nepali_notetaker_project\\nepali_notetaker_package\\DataSet\\asr_nepali_1\\asr_nepali\\data\\16\\1601b4d4ee.flac', 'd:\\nepali_notetaker_project\\nepali_n

In [63]:
from pydub import AudioSegment
from tqdm import tqdm
import os

output_dir = os.path.join(audio_root_dir, 'processed_wav')
os.makedirs(output_dir, exist_ok=True)

wav_paths = []

for i, row in tqdm(df.iterrows(), total=len(df)):
    input_path = row['flac_path']
    file_id = row['file_id']
    output_path = os.path.join(output_dir, file_id + '.wav')
    try:
        # Load and convert to mono 16kHz
        sound = AudioSegment.from_file(input_path, format='flac')
        sound = sound.set_frame_rate(16000).set_channels(1)
        sound.export(output_path, format='wav')
        wav_paths.append(output_path)
    except Exception as e:
        print(f"Failed to process {input_path}: {e}")
        wav_paths.append(None)

df['wav_path'] = wav_paths
df = df[df['wav_path'].notnull()]

# Save only required columns
final_df = df[['wav_path', '2']]
final_df
final_df.columns = ['path', '2']
final_df
final_df.to_csv('final_nepali_dataset_1_13.csv', index=False)

print("✅ Conversion complete. CSV saved as final_nepali_dataset.csv.")


100%|██████████| 613/613 [02:07<00:00,  4.82it/s]

✅ Conversion complete. CSV saved as final_nepali_dataset.csv.
